## アンサンブル学習

学習器の組み合わせて学習する。
一つ一つの学習器を弱学習器という。

In [1]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

digits = datasets.load_digits()
X,y = digits.images, digits.target
X = X.reshape((X.shape[0],-1))

In [2]:
# 学習の流れを関数にしておく
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def classification(clf,x,y,name):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    
    sc = StandardScaler()
    sc.fit(x_train)
    x_train_std = sc.transform(x_train)
    x_test_std = sc.transform(x_test)
    
    clf.fit(x_train_std,y_train)
    y_pred=clf.predict(x_test_std)
    print('%s: 誤分類サンプル数: %d' % (name, (y_test != y_pred).sum()))
    print('%s: 正解率: %.2f' % (name, accuracy_score(y_test, y_pred)))

### ランダムフォレスト

サンプル、特徴量をランダムを復元抽出、決定木を用いて弱学習器を学習させる。

In [3]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(criterion='entropy', n_estimators=100)
classification(forest,X,y,'ランダムフォレスト')

ランダムフォレスト: 誤分類サンプル数: 15
ランダムフォレスト: 正解率: 0.97


### バギング

サンプルを復元抽出する。決定木以外の学習モデルも使ってできる。

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy', max_depth=None)
bag = BaggingClassifier(base_estimator=tree, n_estimators=100, max_samples=1.0,
                      max_features=1.0, bootstrap=True, bootstrap_features=False)
classification(bag,X,y,'バギング')

バギング: 誤分類サンプル数: 25
バギング: 正解率: 0.95


### ブースティング

サンプルを非復元抽出する。
前ステップで誤分類したサンプルの重みをつけて次のステップの弱学習器を学習する。
最終的に重み付きの多数決。

In [5]:
#ブースティング
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(tree, n_estimators=100)
classification(clf,X,y,'ブースティング')

ブースティング: 誤分類サンプル数: 72
ブースティング: 正解率: 0.87


In [6]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0)
classification(clf,X,y,'ブースティング')

ブースティング: 誤分類サンプル数: 444
ブースティング: 正解率: 0.18
